<a href="https://colab.research.google.com/github/imraunav/Pytorch-tutorial/blob/main/0_Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source:
https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

- `torch.utils.data.Dataset` is an object that stores the samples and their corresponding labels.
- `torch.utils.data.Dataloader` is a wraper that wraps an iterable around the `Dataset`.

In [3]:
# Downloading training data from open datasets.
training_data = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Downloading test data from open datasets.
test_data = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=ToTensor(),
)


100%|██████████| 26421880/26421880 [00:00<00:00, 115882325.79it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 3515302.21it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 60767305.67it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw


100%|██████████| 5148/5148 [00:00<00:00, 7093389.29it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [4]:
batch_size = 64

# create dataloaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])


# Creating Models

In [5]:
# Get gpu, mps or cpu for training
if torch.cuda.is_available():
  print("CUDA is available")
  device = "cuda"
elif torch.backend.mps_is_available():
  print("MPS is available")
  device = "mps"
else:
  print("CPU is available")
  device = "cpu"

print(f"Using {device} device")

CUDA is available
Using cuda device


In [6]:
# Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(in_features=28*28, out_features=512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100==0:
      loss, current = loss.item(), (batch+1)*len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()

  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test error: \nAccuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-----------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------------------
loss: 2.307318 [   64/60000]
loss: 2.291116 [ 6464/60000]
loss: 2.272043 [12864/60000]
loss: 2.266589 [19264/60000]
loss: 2.245568 [25664/60000]
loss: 2.223577 [32064/60000]
loss: 2.237163 [38464/60000]
loss: 2.204836 [44864/60000]
loss: 2.195288 [51264/60000]
loss: 2.170021 [57664/60000]
Test error: 
Accuracy: 47.7%, Avg loss: 2.159503 

Epoch 2
-----------------------------------
loss: 2.169707 [   64/60000]
loss: 2.160262 [ 6464/60000]
loss: 2.097749 [12864/60000]
loss: 2.118364 [19264/60000]
loss: 2.069719 [25664/60000]
loss: 2.007324 [32064/60000]
loss: 2.049283 [38464/60000]
loss: 1.967917 [44864/60000]
loss: 1.973882 [51264/60000]
loss: 1.908480 [57664/60000]
Test error: 
Accuracy: 54.9%, Avg loss: 1.899913 

Epoch 3
-----------------------------------
loss: 1.933836 [   64/60000]
loss: 1.905414 [ 6464/60000]
loss: 1.778914 [12864/60000]
loss: 1.828263 [19264/60000]
loss: 1.727929 [25664/60000]
loss: 1.665325 [32064/60000]
loss: 1.7054

# Saving the model

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
